# Data Cleaning

## Reading and exploring the Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,30)

In [3]:
df = pd.read_csv("bengaluru_house_prices.csv")
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [4]:
df.describe()

,bath,balcony,price
count,13247.000000,12711.000000,13320.000000
mean,2.692610,1.584376,112.565627
std,1.341458,0.817263,148.971674
min,1.000000,0.000000,8.000000
25%,2.000000,1.000000,50.000000
50%,2.000000,2.000000,72.000000
75%,3.000000,2.000000,120.000000
max,40.000000,3.000000,3600.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


## Data Cleaning

In [6]:
df.duplicated().any()

True

In [7]:
df = df.drop_duplicates()

In [8]:
df.duplicated().any()

False

In [9]:
df.isnull().any()

area_type       False
availability    False
location         True
size             True
society          True
total_sqft      False
bath             True
balcony          True
price           False
dtype: bool

In [10]:
df["area_type"].value_counts()

Super built-up  Area    8317
Built-up  Area          2398
Plot  Area              1989
Carpet  Area              87
Name: area_type, dtype: int64

In [11]:
df.groupby("area_type").agg("count")

,availability,location,size,society,total_sqft,bath,balcony,price
area_type,,,,,,,,
Built-up Area,2398,2398,2398,1209,2398,2390,2290,2398
Carpet Area,87,87,87,54,87,87,82,87
Plot Area,1989,1989,1973,310,1989,1973,1801,1989
Super built-up Area,8317,8316,8317,5890,8317,8268,8013,8317


In [12]:
df1 = df.drop(["area_type", "availability", "society", "balcony"], axis = "columns")
df1.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [13]:
df1.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [14]:
df1.columns[df1.isna().any()]

Index(['location', 'size', 'bath'], dtype='object')

In [15]:
df1["size"] = df1["size"].fillna(method= "ffill")

In [16]:
df1.bath = df1.bath.fillna(df1.bath.mean())

In [17]:
df1["location"] = df1["location"].fillna(method= "ffill")

In [22]:
df1.isna().any()

location      False
size          False
total_sqft    False
bath          False
price         False
dtype: bool

In [23]:
df1.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

### Examine locations which is a categorical variable. 

In [24]:
len(df1.location.unique())

1305

In [25]:
locations = df1.groupby("location")["location"].count().sort_values(ascending=False)
locations

location
Whitefield                              523
Sarjapur  Road                          380
Electronic City                         287
Kanakpura Road                          249
Thanisandra                             229
                                       ... 
Kodanda Reddy Layout                      1
Kirloskar layout, Basaveshwarnagar        1
Kengeri Satellite Town Stage II           1
Kengeri Satellite Town KHB Apartment      1
 Anekal                                   1
Name: location, Length: 1305, dtype: int64

### Fixing whitespace and Capitalization

In [26]:
df1["location"] = df1.location.apply(lambda x: x.strip())

In [27]:
df1["location"] = df1.location.apply(lambda x: x.lower())

## Collapsing categories into one

In [34]:
len(locations[locations<=10])

1072

In [35]:
location_lessthan10 = locations[locations<=10]


#### Any location having less than 10 data points should be tagged as "other" location. This way number of categories can be reduced by huge amount. Later on when we do one hot encoding, it will help us with having fewer dummy columns

In [36]:
df1.location = df1.location.apply(lambda x : "other" if x in location_lessthan10 else x)

In [37]:
len(df1.location.unique())

1250

In [38]:
df1.head(10)

,location,size,total_sqft,bath,price
0,electronic city phase ii,2 BHK,1056,2.0,39.07
1,chikka tirupathi,4 Bedroom,2600,5.0,120.00
2,uttarahalli,3 BHK,1440,2.0,62.00
3,lingadheeranahalli,3 BHK,1521,3.0,95.00
4,kothanur,2 BHK,1200,2.0,51.00
5,whitefield,2 BHK,1170,2.0,38.00
6,old airport road,4 BHK,2732,4.0,204.00
7,rajaji nagar,4 BHK,3300,4.0,600.00
8,marathahalli,3 BHK,1310,3.0,63.25
9,gandhi bazar,6 Bedroom,1020,6.0,370.00


In [39]:
df1.shape

(12791, 5)